In [ ]:
!git clone https://github.com/punyajoy/HateXplain.git

In [ ]:
import warnings
warnings.filterwarnings('ignore')


In [ ]:
cd HateXplain/

/content/HateXplain


In [ ]:
!mkdir Saved/
!mkdir explanations_dicts/

In [ ]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip  -P Data/

--2022-10-22 13:22:37--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2022-10-22 13:22:37--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2022-10-22 13:22:38--  https://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]


In [ ]:
!unzip Data/glove.42B.300d.zip -d Data/

Archive:  Data/glove.42B.300d.zip
  inflating: Data/glove.42B.300d.txt  


In [ ]:
!rm Data/glove.42B.300d.zip

In [ ]:
!pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 26.1 MB 1.3 MB/s 
     |████████████████████████████████| 59 kB 5.4 MB/s 
     |████████████████████████████████| 377 kB 75.4 MB/s 
     |████████████████████████████████| 80 kB 9.5 MB/s 
     |████████████████████████████████| 10.0 MB 24.4 MB/s 
     |████████████████████████████████| 499 kB 64.0 MB/s 
     |████████████████████████████████| 275 kB 68.6 MB/s 
     |████████████████████████████████| 17.3 MB 20.1 MB/s 
     |████████████████████████████████| 12.4 MB 53.9 MB/s 
     |████████████████████████████████| 24.2 MB 1.3 MB/s 
     |████████████████████████████████| 86 kB 6.9 MB/s 
     |████████████████████████████████| 676.9 MB 4.0 kB/s 
     |████████████████████████████████| 6.8 MB 51.2 MB/s 
     |████████████████████████████████| 50 kB 6.8 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
     |███████████████████████████████

In [ ]:
!pip install -U spacy
! pip install keras==2.8
!pip install tensorflow==2.8

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 34.6 MB/s 
  Attempting uninstall: spacy
    Found existing installation: spacy 3.4.1
    Uninstalling spacy-3.4.1:
      Successfully uninstalled spacy-3.4.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.4 MB 30.2 MB/s 
  Attempting uninstall: keras
    Found existing installation: keras 2.9.0
    Uninstalling keras-2.9.0:
      Successfully uninstalled keras-2.9.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires keras<2.10.0,>=2.9.0rc0, but you have keras 2.8.0 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |███████████

In [ ]:
from gensim.models import KeyedVectors
from gensim.test.utils import get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
glove2word2vec('Data/glove.42B.300d.txt', 'Data/glove.42B.300d_w2v.txt')
word2vecmodel1 = KeyedVectors.load_word2vec_format('Data/glove.42B.300d_w2v.txt', binary=False)
word2vecmodel1.save("Data/word2vec.model")

In [ ]:
import gc
del word2vecmodel1
gc.collect()

236

In [ ]:
!rm Data/glove.42B.300d.txt
!rm Data/glove.42B.300d_w2v.txt

In [ ]:
from manual_training_inference import *

Word statistics files not found!
Downloading... done!
Unpacking... done!
Reading twitter - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_1grams.txt
Reading twitter - 2grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/twitter/counts_2grams.txt
Reading english - 1grams ...
generating cache file for faster loading...
reading ngrams /root/.ekphrasis/stats/english/counts_1grams.txt


In [ ]:
path_file='best_model_json/bestModel_birnnscrat.json'
with open(path_file,mode='r') as f:
    params = json.load(f)
for key in params:
    if params[key] == 'True':
          params[key]=True
    elif params[key] == 'False':
          params[key]=False
    if( key in ['batch_size','num_classes','hidden_size','supervised_layer_pos','num_supervised_heads','random_seed','max_length']):
        if(params[key]!='N/A'):
            params[key]=int(params[key])
        
    if((key == 'weights') and (params['auto_weights']==False)):
        params[key] = ast.literal_eval(params[key])

##### change in logging to output the results to neptune
params['logging']='local'
params['device']='cpu'
params['best_params']=False

if torch.cuda.is_available() and params['device']=='cuda':    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
else:
    print('Since you dont want to use GPU, using the CPU instead.')
    device = torch.device("cpu")
    
    
#### Few handy keys that you can directly change.
params['variance']=1
params['epochs']=5
params['to_save']=True
params['num_classes']=2
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']
if(params['num_classes']==2 and (params['auto_weights']==False)):
      params['weights']=[1.0,1.0]
        
#for att_lambda in [0.001,0.01,0.1,1,10,100]
train_model(params,device)

Since you dont want to use GPU, using the CPU instead.


  0%|          | 64/20148 [00:00<00:31, 631.34it/s]

total_data 20148


100%|██████████| 20148/20148 [00:32<00:00, 615.32it/s]


attention_error: 0
no_majority: 919


...

61it [00:04, 14.84it/s]
2it [00:00, 15.28it/s]

 Accuracy: 0.69
 Fscore: 0.69
 Precision: 0.71
 Recall: 0.71
 Roc Auc: 0.00
 Test took: 0:00:04
model previously passed
Running eval on  test ...


61it [00:04, 14.68it/s]


 Accuracy: 0.68
 Fscore: 0.68
 Precision: 0.70
 Recall: 0.71
 Roc Auc: 0.00
 Test took: 0:00:04
best_val_fscore 0.6981002490464414
best_test_fscore 0.697965467609571
best_val_rocauc 0
best_test_rocauc 0
best_val_precision 0.73181994261365
best_test_precision 0.7287470470600899
best_val_recall 0.7276576357195039
best_test_recall 0.7260706079431697


1

In [ ]:
params['num_classes']=3
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']
if(params['num_classes']==2 and (params['auto_weights']==False)):
      params['weights']=[1.0,1.0]
        
#for att_lambda in [0.001,0.01,0.1,1,10,100]
train_model(params,device)

  9%|▉         | 1404/15383 [00:00<00:01, 7061.43it/s]

unk


  5%|▌         | 792/15383 [00:00<00:01, 7911.14it/s]

(22236, 300)


100%|██████████| 1924/1924 [00:00<00:00, 8321.53it/s]


...

61it [00:04, 14.83it/s]
2it [00:00, 15.09it/s]

 Accuracy: 0.64
 Fscore: 0.63
 Precision: 0.63
 Recall: 0.63
 Roc Auc: 0.79
 Test took: 0:00:04
model previously passed
Running eval on  test ...


61it [00:04, 14.89it/s]


 Accuracy: 0.63
 Fscore: 0.62
 Precision: 0.62
 Recall: 0.62
 Roc Auc: 0.79
 Test took: 0:00:04
best_val_fscore 0.6365513941496782
best_test_fscore 0.6394014618815519
best_val_rocauc 0.8117476454123843
best_test_rocauc 0.8203029594853541
best_val_precision 0.6370146992257849
best_test_precision 0.6387270124237755
best_val_recall 0.6361208687390083
best_test_recall 0.6401887284784715


1

In [ ]:
import gc
gc.collect()

127

In [ ]:
!python testing_with_rational.py birnn_scrat 100
!python testing_for_bias.py birnn_scrat 100

/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/tokenizer.py:225: FutureWarning: Possible nested set at position 2190
  self.tok = re.compile(r"({})".format("|".join(pipeline)))
Reading twitter - 1grams ...
Reading twitter - 2grams ...
/usr/local/lib/python3.7/dist-packages/ekphrasis/classes/exmanager.py:14: FutureWarning: Possible nested set at position 42
  regexes = {k.lower(): re.compile(self.expressions[k]) for k, v in
Reading english - 1grams ...
Since you dont want to use GPU, using the CPU instead.
tcmalloc: large alloc 2300993536 bytes == 0x3ebec000 @  0x7f3ec03471e7 0x4b2150 0x5ac2ec 0x5e3d6d 0x58ee7f 0x590c9f 0x591039 0x4fb96a 0x4fc108 0x4fe054 0x5ef9b8 0x58eb9c 0x51b4e6 0x58f2a7 0x51740e 0x5b4a3e 0x58f49e 0x51837f 0x5b41c5 0x58f49e 0x51837f 0x5b41c5 0x604133 0x606e06 0x606ecc 0x609aa6 0x64d332 0x64d4de 0x7f3ebff44c87 0x5b561a
tcmalloc: large alloc 2300993536 bytes == 0xc7e52000 @  0x7f3ec03471e7 0x4b2150 0x5ac2ec 0x4fc11a 0x4fe054 0x5ef9b8 0x58eb9c 0x51b4e6 0x58f2a

In [ ]:
!ls explanations_dicts

bestModel_birnnscrat_100_explanation_top5.json	bestModel_birnnscrat_bias.json




---

---




**Bias Calculation**

---



In [ ]:
from collections import Counter,defaultdict
from tqdm.notebook import tqdm
import json
import numpy as np

In [ ]:
# get_annotated_data method is used to load the dataset
from Preprocess.dataCollect import get_annotated_data

In [ ]:
dict_data_folder={
      '2':{'data_file':'Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'Data/dataset.json','class_label':'Data/classes.npy'}
}

params = {}

# We need to load the dataset with the labels as 'toxic' and 'non-toxic'. 
# We consider hatespeech and offensive as toxic and normal as non-toxic.
params['num_classes']=2  
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [ ]:
data_all_labelled

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",1,[None],normal,2,[None],normal,3,[None],normal,[],non-toxic
1,1179063826874032128_twitter,"[we, cannot, continue, calling, ourselves, fem...",1,[None],normal,2,[None],normal,3,[None],normal,[],non-toxic
2,1178793830532956161_twitter,"[nawt, yall, niggers, ignoring, me]",4,[African],normal,2,[None],normal,3,[African],hatespeech,[],non-toxic
3,1179088797964763136_twitter,"[<user>, i, am, bit, confused, coz, chinese, p...",1,[Asian],hatespeech,4,[Asian],offensive,3,[Asian],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",toxic
4,1179085312976445440_twitter,"[this, bitch, in, whataburger, eating, a, burg...",4,"[Caucasian, Women]",hatespeech,2,"[Women, Caucasian]",hatespeech,3,"[Women, Caucasian]",offensive,"[[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",toxic
...,...,...,...,...,...,...,...,...,...,...,...,...,...
20143,9989999_gab,"[if, ur, still, on, twitter, tell, carlton, i,...",217,"[Men, Women, Other]",offensive,199,[None],offensive,215,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,...",toxic
20144,9990225_gab,"[when, i, first, got, on, here, and, said, i, ...",220,[African],offensive,223,"[African, Other]",offensive,231,[None],normal,"[[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0,...",toxic
20145,9991681_gab,"[was, macht, der, moslem, wenn, der, zion, geg...",206,[Islam],offensive,203,[Other],normal,211,[None],normal,[],non-toxic
20146,9992513_gab,"[it, is, awful, look, at, world, demographics,...",209,[Hispanic],hatespeech,253,[Asian],hatespeech,222,[Asian],offensive,"[[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1,...",toxic


In [ ]:
def generate_target_information(dataset):
    final_target_output = defaultdict(list)
    all_communities_selected = []
    
    for each in dataset.iterrows(): 
        # All the target communities tagged for this post
        all_targets = each[1]['target1']+each[1]['target2']+each[1]['target3']  
        community_dict = dict(Counter(all_targets))
        
        # Select only those communities which are present more than once.
        for key in community_dict:
            if community_dict[key]>1:  
                final_target_output[each[1]['post_id']].append(key)
                all_communities_selected.append(key)
        
        # If no community is selected based on majority voting then we don't select any community
        if each[1]['post_id'] not in final_target_output:
            final_target_output[each[1]['post_id']].append('None')
            all_communities_selected.append(key)

    return final_target_output, all_communities_selected

In [ ]:
target_information, all_communities_selected = generate_target_information(data_all_labelled)

In [ ]:
community_count_dict = Counter(all_communities_selected)

# We remove None and Other from dictionary
community_count_dict.pop('None')
community_count_dict.pop('Other')

# For the bias calculation, we are considering the top 10 communites based on their count
list_selected_community = [community for community, value in community_count_dict.most_common(10)]
list_selected_community

['African',
 'Islam',
 'Jewish',
 'Homosexual',
 'Women',
 'Refugee',
 'Arab',
 'Caucasian',
 'Asian',
 'Hispanic']

In [ ]:
# Based on the top 10 communities, we filter the target_information
# This will remove the other communities from the calculation

final_target_information ={}
for each in target_information:
    temp = list(set(target_information[each])&set(list_selected_community))
    if len(temp) == 0:
        final_target_information[each] = None
    else:
        final_target_information[each] = temp

In [ ]:
# Add a new column 'final_target_category' which will contain the selected target community names
data_all_labelled['final_target_category'] = data_all_labelled['post_id'].map(final_target_information)

In [ ]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
postpost_id_divisions_path = './Data/post_id_divisions.json'

with open(postpost_id_divisions_path, 'r') as fp:
    post_id_dict=json.load(fp)

data_all_labelled_bias = data_all_labelled[data_all_labelled['post_id'].isin(post_id_dict['test'])]
data_all_labelled_bias

,post_id,text,annotatorid1,target1,label1,annotatorid2,target2,label2,annotatorid3,target3,label3,rationales,final_label,final_target_category
0,1179055004553900032_twitter,"[i, dont, think, im, getting, my, baby, them, ...",1,[None],normal,2,[None],normal,3,[None],normal,[],non-toxic,None
5,13851720_gab,"[laura, loomer, raped, me, while, screaming, a...",1,[Jewish],hatespeech,2,[Jewish],hatespeech,3,[Jewish],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0,...",toxic,[Jewish]
17,1178818409812746240_twitter,"[<user>, what, did, the, old, lady, do, was, s...",9,[None],normal,10,[None],normal,4,[None],normal,[],non-toxic,None
25,19346774_gab,"[as, much, as, i, appreciate, a, format, of, o...",9,[None],normal,13,[None],normal,4,[Hispanic],offensive,[],non-toxic,None
36,1165819983701643266_twitter,"[sex, be, so, good, a, bitch, be, slow, stroki...",4,[Women],offensive,7,[Women],offensive,16,[None],normal,"[[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0], [0, 0, 0, ...",toxic,[Women]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20105,9773208_gab,"[it, always, women, trying, this, shit, like, ...",200,[Women],hatespeech,202,[Women],offensive,203,[Women],offensive,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",toxic,[Women]
20109,9802943_gab,"[because, women, would, never, lie, about, bei...",228,[Women],offensive,222,[Women],offensive,209,[Women],normal,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",toxic,[Women]
20117,9826963_gab,"[how, is, all, that, awesome, muslim, diversit...",228,[Islam],offensive,222,[Islam],offensive,209,[Islam],offensive,"[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,...",toxic,[Islam]
20128,9867117_gab,"[well, my, dear, lgbtq, brothers, and, sisters...",207,"[Islam, Other]",offensive,223,[Islam],hatespeech,231,[Islam],hatespeech,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",toxic,[Islam]


In [ ]:
from sklearn.metrics import roc_auc_score

# The names of the files which will contain the bias output for each model
bias_score_file_mapping={
    'BiRNN-Attn':'bestModel_birnnscrat_bias.json',
}

# The parent folder path of the bias output files
parent_path = './explanations_dicts/'

# The bias methods that will be considered
method_list = ['subgroup', 'bpsn', 'bnsp']

community_list = list(list_selected_community)

In [ ]:
# This function is used to convert the classification into a [0-1] score 
# with a value of 0 meaning non-toxic and 1 meaning toxic
def convert_to_score(label_name, label_dict):
    if label_name=='non-toxic':
        return 1-label_dict[label_name]
    else:
        return label_dict[label_name]

In [ ]:
# Function to divide the ids into postive or class class based on the method.
def bias_evaluation_metric(dataset, method, community):
    positive_ids = []
    negative_ids = []
    if method=='subgroup':
        for eachrow in dataset.iterrows():
            if eachrow[1]['final_target_category'] == None:
                continue
            if community in eachrow[1]['final_target_category']:
                if eachrow[1]['final_label'] =='non-toxic':
                    negative_ids.append(eachrow[1]['post_id'])
                else:
                    positive_ids.append(eachrow[1]['post_id'])
            else:
                pass
    elif method=='bpsn':
        for eachrow in dataset.iterrows():
            if eachrow[1]['final_target_category'] == None:
                continue
            if community in eachrow[1]['final_target_category']:
#                 print(eachrow[1]['final_label'])
                if eachrow[1]['final_label'] =='non-toxic':
                    negative_ids.append(eachrow[1]['post_id'])
                else:
                    pass
            else:
                if eachrow[1]['final_label'] !='non-toxic':
                    positive_ids.append(eachrow[1]['post_id'])
                else:
                    pass
    elif method=='bnsp':
        for eachrow in dataset.iterrows():
            if eachrow[1]['final_target_category'] == None:
                continue
            if community in eachrow[1]['final_target_category']:
                if eachrow[1]['final_label'] !='non-toxic':
                    positive_ids.append(eachrow[1]['post_id'])
                else:
                    pass
            else:
                if eachrow[1]['final_label'] =='non-toxic':
                    negative_ids.append(eachrow[1]['post_id'])
                else:
                    pass
    else:
        print('Incorrect option selected!!!')
                
    return {'positiveID':positive_ids, 'negativeID':negative_ids}

In [ ]:
final_bias_dictionary = defaultdict(lambda: defaultdict(dict))

# We load each of the model bias output file and compute the bias score using each method for all the community
for each_model in tqdm(bias_score_file_mapping):
    total_data ={}
    with open(parent_path+bias_score_file_mapping[each_model]) as fp:
        for line in fp:
            data = json.loads(line)
            total_data[data['annotation_id']] = data
    for each_method in method_list:
        for each_community in community_list:
            community_data = bias_evaluation_metric(data_all_labelled_bias, each_method, each_community)
            truth_values = []
            prediction_values = []


            label_to_value = {'toxic':1.0, 'non-toxic':0.0}
            for each in community_data['positiveID']:
                truth_values.append(label_to_value[total_data[each]['ground_truth']])
                prediction_values.append(convert_to_score(total_data[each]['classification'], total_data[each]['classification_scores']))

            for each in community_data['negativeID']:
                truth_values.append(label_to_value[total_data[each]['ground_truth']])
                prediction_values.append(convert_to_score(total_data[each]['classification'], total_data[each]['classification_scores']))

            roc_output_value = roc_auc_score(truth_values, prediction_values)
            final_bias_dictionary[each_model][each_method][each_community] = roc_output_value

In [ ]:
%precision 4

'%.4f'

In [ ]:
# To combine the per-identity Bias AUCs into one overall measure, we calculate their generalized mean as defined below:
power_value = -5
num_communities = len(community_list)

for each_model in final_bias_dictionary:
    for each_method in final_bias_dictionary[each_model]:
        temp_value =[]
        for each_community in final_bias_dictionary[each_model][each_method]:
            temp_value.append(pow(final_bias_dictionary[each_model][each_method][each_community], power_value))
        print(each_model, each_method, pow(np.sum(temp_value)/num_communities, 1/power_value))

BiRNN-Attn subgroup 0.7178253973318881
BiRNN-Attn bpsn 0.6412719177169195
BiRNN-Attn bnsp 0.706659682390115




---


**Explainability Calculation**

---



In [ ]:
import json
from tqdm.notebook import tqdm
import more_itertools as mit
import os

In [ ]:
# get_annotated_data method is used to load the dataset
from Preprocess import *
from Preprocess.dataCollect import *

In [ ]:
dict_data_folder={
      '2':{'data_file':'Data/dataset.json','class_label':'Data/classes_two.npy'},
      '3':{'data_file':'Data/dataset.json','class_label':'Data/classes.npy'}
}

# We need to load the dataset with the labels as 'hatespeech', 'offensive', and 'normal' (3-class). 

params = {}
params['num_classes']=3
params['data_file']=dict_data_folder[str(params['num_classes'])]['data_file']
params['class_names']=dict_data_folder[str(params['num_classes'])]['class_label']

data_all_labelled=get_annotated_data(params)

In [ ]:
# The important key here is the 'bert_token'. Set it to True for Bert based models and False for Others.

params_data={
    'include_special':False,  #True is want to include <url> in place of urls if False will be removed
    'bert_tokens':False, #True /False
    'type_attention':'softmax', #softmax
    'set_decay':0.1,
    'majority':2,
    'max_length':128,
    'variance':10,
    'window':4,
    'alpha':0.5,
    'p_value':0.8,
    'method':'additive',
    'decay':False,
    'normalized':False,
    'not_recollect':True,
}


if(params_data['bert_tokens']):
    print('Loading BERT tokenizer...')
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=False)
else:
    print('Loading Normal tokenizer...')
    tokenizer=None

Loading Normal tokenizer...


In [ ]:
# Load the whole dataset and get the tokenwise rationales
def get_training_data(data):
    post_ids_list=[]
    text_list=[]
    attention_list=[]
    label_list=[]
    
    final_binny_output = []
    print('total_data',len(data))
    for index,row in tqdm(data.iterrows(),total=len(data)):
        annotation=row['final_label']
        
        text=row['text']
        post_id=row['post_id']
        annotation_list=[row['label1'],row['label2'],row['label3']]
        tokens_all = list(row['text'])
#         attention_masks =  [list(row['explain1']),list(row['explain2']),list(row['explain1'])]
        
        if(annotation!= 'undecided'):
            tokens_all,attention_masks=returnMask(row, params_data, tokenizer)
            final_binny_output.append([post_id, annotation, tokens_all, attention_masks, annotation_list])

    return final_binny_output

In [ ]:
training_data=get_training_data(data_all_labelled)

  0%|          | 65/20148 [00:00<00:31, 638.99it/s]

total_data 20148


100%|██████████| 20148/20148 [00:29<00:00, 683.25it/s]


In [ ]:
# https://stackoverflow.com/questions/2154249/identify-groups-of-continuous-numbers-in-a-list
def find_ranges(iterable):
    """Yield range of consecutive numbers."""
    for group in mit.consecutive_groups(iterable):
        group = list(group)
        if len(group) == 1:
            yield group[0]
        else:
            yield group[0], group[-1]
            
# Convert dataset into ERASER format: https://github.com/jayded/eraserbenchmark/blob/master/rationale_benchmark/utils.py
def get_evidence(post_id, anno_text, explanations):
    output = []

    indexes = sorted([i for i, each in enumerate(explanations) if each==1])
    span_list = list(find_ranges(indexes))

    for each in span_list:
        if type(each)== int:
            start = each
            end = each+1
        elif len(each) == 2:
            start = each[0]
            end = each[1]+1
        else:
            print('error')

        output.append({"docid":post_id, 
              "end_sentence": -1, 
              "end_token": end, 
              "start_sentence": -1, 
              "start_token": start, 
              "text": ' '.join([str(x) for x in anno_text[start:end]])})
    return output

# To use the metrices defined in ERASER, we will have to convert the dataset
def convert_to_eraser_format(dataset, method, save_split, save_path, id_division):  
    final_output = []
    
    if save_split:
        train_fp = open(save_path+'train.jsonl', 'w')
        val_fp = open(save_path+'val.jsonl', 'w')
        test_fp = open(save_path+'test.jsonl', 'w')
            
    for tcount, eachrow in enumerate(dataset):
        
        temp = {}
        post_id = eachrow[0]
        post_class = eachrow[1]
        anno_text_list = eachrow[2]
        majority_label = eachrow[1]
        
        if majority_label=='normal':
            continue
        
        all_labels = eachrow[4]
        explanations = []
        for each_explain in eachrow[3]:
            explanations.append(list(each_explain))
        
        # For this work, we have considered the union of explanations. Other options could be explored as well.
        if method == 'union':
            final_explanation = [any(each) for each in zip(*explanations)]
            final_explanation = [int(each) for each in final_explanation]
        
            
        temp['annotation_id'] = post_id
        temp['classification'] = post_class
        temp['evidences'] = [get_evidence(post_id, list(anno_text_list), final_explanation)]
        temp['query'] = "What is the class?"
        temp['query_type'] = None
        final_output.append(temp)
        
        if save_split:
            if not os.path.exists(save_path+'docs'):
                os.makedirs(save_path+'docs')
            
            with open(save_path+'docs/'+post_id, 'w') as fp:
                fp.write(' '.join([str(x) for x in list(anno_text_list)]))
            
            if post_id in id_division['train']:
                train_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['val']:
                val_fp.write(json.dumps(temp)+'\n')
            
            elif post_id in id_division['test']:
                test_fp.write(json.dumps(temp)+'\n')
            else:
                print(post_id)
    
    if save_split:
        train_fp.close()
        val_fp.close()
        test_fp.close()
        
    return final_output

In [ ]:
# The post_id_divisions file stores the train, val, test split ids. We select only the test ids.
with open('./Data/post_id_divisions.json') as fp:
    id_division = json.load(fp)

In [ ]:
!mkdir ./Data/Evaluation
!mkdir ./Data/Evaluation/Model_Eval

In [ ]:
method = 'union'
save_split = True
save_path = './Data/Evaluation/Model_Eval/'  #The dataset in Eraser Format will be stored here.
output_eraser = convert_to_eraser_format(training_data, method, save_split, save_path, id_division)

In [ ]:
!ls Data/Evaluation/Model_Eval/

docs  test.jsonl  train.jsonl  val.jsonl


In [ ]:
!ls

best_model_json			     Models
best_runs.sh			     Parameters_description.md
Bias_Calculation_NB.ipynb	     parameters_selection.py
convert_to_word2vec.py		     Preprocess
Data				     __pycache__
eraserbenchmark			     README.md
Example_HateExplain.ipynb	     requirements.txt
Explainability_Calculation_NB.ipynb  Saved
explanations_dicts		     TensorDataset
Figures				     testing_for_bias.py
LICENSE				     testing_with_lime.py
manual_training_inference.py	     testing_with_rational.py
model_explain_output.json	     test_parallel.sh


In [ ]:
cd eraserbenchmark/

/content/HateXplain/eraserbenchmark


In [ ]:
!ls 

data_exploration.ipynb	params		     README.md	       requirements.txt
LICENSE			rationale_benchmark  REPRODUCTION.txt


In [ ]:
!PYTHONPATH=./:$PYTHONPATH python rationale_benchmark/metrics.py --split test  --data_dir ../Data/Evaluation/Model_Eval --results ../explanations_dicts/bestModel_birnnscrat_100_explanation_top5.json --score_file ../model_explain_output.json

  1717 MainThread Error in instances: 0 instances fail validation: set()
  3234 MainThread No sentence level predictions detected, skipping sentence-level diagnostic
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
{'classification_scores': {'accuracy': 0.6217162872154116,
                           'aopc_thresholds': None,
                           'comprehensiveness': 0.2899375320616098,
                           'comprehensiveness_aopc': None,
                           'comprehensiveness_aopc_points': None,
                           'comprehensiveness_entropy': 0.06618782785757517,
                           'comprehensiveness_kl': 0.6169633591014108,
                           'prf': {'accuracy': 0.6217162872154116,
            

In [ ]:
# print the required results
with open('../model_explain_output.json') as fp:
    output_data = json.load(fp)

print('\nPlausibility')
print('IOU F1 :', output_data['iou_scores'][0]['macro']['f1'])
print('Token F1 :', output_data['token_prf']['instance_macro']['f1'])
print('AUPRC :', output_data['token_soft_metrics']['auprc'])

print('\nFaithfulness')
print('Comprehensiveness :', output_data['classification_scores']['comprehensiveness'])
print('Sufficiency', output_data['classification_scores']['sufficiency'])


Plausibility
IOU F1 : 0.22809206487116668
Token F1 : 0.5074295479278655
AUPRC : 0.8384250389403275

Faithfulness
Comprehensiveness : 0.2899375320616098
Sufficiency 0.0014583442993005395
